<a href="https://colab.research.google.com/github/jsopesens/Data-Curation-SCRIPT/blob/main/Convert_Excel_into_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is designed to convert a file xlsx or xls into a CSV file.
In case that the original excel file contains more than one sheet, this code will generate as much csv files as original sheets.
This code is specially usefull in non-english countries due to the fact that manual convertions results in ';' separators instead of ','.


---

To test this script, go to the left navbar and click archives and go to 'content'>'sample_data'>'convert_excel_to_csv'. Place your excel file here.

---


If original file contains only one sheet, the name of the result csv will be "original_file_name".csv.
In case of several sheets, the name will be "original_file_name"_"sheet_name".csv

1. Import of the needed libraries

In [ ]:
import os
import pandas as pd


2. Initialize the global variables

In [ ]:
FOLDER = 'sample_data/convert_excel_to_csv'

3. Find the required file

In [ ]:
def find_xls_file() -> str:
    for file in os.listdir(FOLDER):
        if file.endswith('.xls') or file.endswith('.xlsx'):
            return file
    raise FileNotFoundError("No file with .xls or .xlsx extension found in the folder.")

xls_file = find_xls_file()

4. Evaluate content of the file

In [ ]:
def evaluate_engine(file: str) -> str:
    file_extension = os.path.splitext(file)[1]

    if file_extension == '.xlsx':
        engine = 'openpyxl'
    elif file_extension == '.xls':
        engine = 'xlrd'
    else:
        raise ValueError("File format not supported. Must be .xls or .xlsx.")

    return engine


def get_xls_active_sheet(xls_file: str) -> list:
    try:
        engine = evaluate_engine(f"{FOLDER}/{xls_file}")
        excel_file = pd.ExcelFile(f"{FOLDER}/{xls_file}", engine=engine)
        sheets_with_content = []

        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            if not df.empty:
                sheets_with_content.append({
                    'name': sheet_name,
                    'content': df
                })

        if not sheets_with_content:
            raise ValueError("Excel file does not contain sheets with data.")

        return sheets_with_content
    except Exception as e:
        raise RuntimeError(f"Error reading Excel file: {e}")


xls_sheets = get_xls_active_sheet(xls_file)

5. Convert content into new files

In [ ]:
def convert_to_csv(xls_file: str, sheets: list) -> None:
    base_name = os.path.splitext(f"{FOLDER}/{xls_file}")[0]

    # If only one sheet on the original file
    if len(sheets) == 1:
        csv_file = f"{base_name}.csv"
        sheets[0]['content'].to_csv(csv_file, index=False, encoding='utf-8')
    # If multiple sheets on the original file
    if len(sheets) > 1:
        for sheet in sheets:
            sheet_name = sheet['name']
            csv_file = f"{base_name}_{sheet_name}.csv"
            sheet['content'].to_csv(csv_file, index=False, encoding='utf-8')

convert_to_csv(xls_file, xls_sheets)